# dType Data Balancing on Shards

In [1]:
import random

In [24]:
shard_count = 20
dtype_count = 50
max_shard_load = 400
max_dtype_load = 2000
average_coef = 1.3

In [25]:
shard_loads = list(enumerate([random.randrange(i, max_shard_load) for i in range(shard_count)]))
dtype_loads = list(enumerate([random.randrange(i, max_dtype_load) for i in range(dtype_count)]))
shards = [[] for i in range(shard_count)]

print('dtype_loads', dtype_loads)
print('shard_loads', shard_loads)

dtype_loads [(0, 1230), (1, 1348), (2, 893), (3, 942), (4, 667), (5, 1438), (6, 113), (7, 583), (8, 1894), (9, 495), (10, 1918), (11, 1805), (12, 400), (13, 1849), (14, 1001), (15, 1309), (16, 354), (17, 1169), (18, 1567), (19, 195), (20, 1649), (21, 1081), (22, 1928), (23, 469), (24, 1633), (25, 714), (26, 353), (27, 786), (28, 398), (29, 787), (30, 882), (31, 324), (32, 209), (33, 1308), (34, 1126), (35, 1935), (36, 1994), (37, 1565), (38, 1739), (39, 1335), (40, 1685), (41, 1994), (42, 1430), (43, 310), (44, 459), (45, 322), (46, 1159), (47, 1648), (48, 1789), (49, 1949)]
shard_loads [(0, 384), (1, 121), (2, 74), (3, 189), (4, 397), (5, 59), (6, 258), (7, 21), (8, 16), (9, 197), (10, 220), (11, 100), (12, 275), (13, 255), (14, 303), (15, 58), (16, 380), (17, 70), (18, 296), (19, 301)]


In [26]:
# Calculate average count per shard
average_load_shard = (sum(i[1] for i in dtype_loads) + sum(i[1] for i in shard_loads)) / shard_count
average_load_shard *= average_coef

print('average_load_shard', average_load_shard)

average_load_shard 3906.7599999999998


In [27]:
next_index_s = 0
next_index_dt = 0
last_index_dt = len(dtype_loads) - 1

# Sort first
dtype_loads.sort(key=lambda tup: tup[1], reverse=True)
shard_loads.sort(key=lambda tup: tup[1])

# Move heavier than average dtypes on the least heaviest shards
for i, dload in dtype_loads:
    if dload >= average_load_shard:
        shards[next_index_s].append(i)
        next_index_s += 1
        next_index_dt += 1


# 
for i, dload in dtype_loads[next_index_dt:]:
    shards[next_index_s].append(i)
    load = shard_loads[next_index_s][1] + dload + dtype_loads[last_index_dt][1]
    while last_index_dt > next_index_dt and load <= average_load_shard:
        shards[next_index_s].append(dtype_loads[last_index_dt][0])
        last_index_dt -= 1
        load += dtype_loads[last_index_dt][1]
    next_index_s += 1
    next_index_dt += 1
    if next_index_dt > last_index_dt:
        break

print('(shard_index, shard_load, dtype_indexes)')
final_shards = [(shard_loads[x][0], sum(shards[x]), shards[x]) for x, _ in enumerate(shards)]
print('final_shards', final_shards)

(shard_index, shard_load, dtype_indexes)
final_shards [(8, 238, [36, 6, 19, 32, 43, 45, 31, 26]), (7, 141, [41, 16, 28, 12, 44]), (15, 88, [49, 23, 9, 7]), (5, 64, [35, 4, 25]), (17, 78, [22, 27, 29]), (2, 42, [10, 30, 2]), (11, 11, [8, 3]), (1, 27, [13, 14]), (3, 32, [11, 21]), (9, 82, [48, 34]), (10, 84, [38, 46]), (13, 57, [40, 17]), (6, 20, [20, 0]), (12, 80, [47, 33]), (18, 39, [24, 15]), (19, 57, [18, 39]), (14, 38, [37, 1]), (16, 47, [5, 42]), (0, 0, []), (4, 0, [])]
